# elaws-openapi-proxy example usage by langchain

## Prerequisites

1. This example uses the [OpenAI API](https://openai.com/blog/openai-api). Please prepare your OpenAI API key. Be aware that calling the API may incur costs.

2. Get ready your [elaws-openapi-proxy server](../../README.md) and make sure the OpenAPI spec of the server is accessible from the environment this notebook runs.

3. The Python environment of this notebook is managed by [Poetry](https://python-poetry.org/docs/#installation). Please install Poetry if necessary, and run `poetry install` in the same directory as this notebook.

## How to run the notebook

1. Save the OpenAI API key in the [`.env`](.env) file in the same directory as this notebook. Write the key in the `.env` file as below.
    ```
    OPENAI_API_KEY=sk-*******************************
    ```

2. Before running the notebook, make sure the [elaws-openapi-proxy server](../../README.md) is running. If necessary, please modify the `ELAWS_OPENAPI_SPEC_URI` variable in the next cell.

3. Run the all cells below.

In [1]:
"""
The OpenAPI spec URI of the elaws-openapi-proxy server.
Please modify it if necessary.
"""
ELAWS_OPENAPI_SPEC_URI = "http://localhost:3000/openapi.json"

In [2]:
DEBUG = False

from dotenv import load_dotenv
assert load_dotenv()
import langchain
print(f"{langchain.__version__ = }")
langchain.verbose = True
if DEBUG:
    langchain.debug = True

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, OpenAIFunctionsAgent, AgentExecutor
from langchain.utilities.openapi import OpenAPISpec
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.tools import format_tool_to_openai_function
from langchain.schema.messages import SystemMessage

from IPython.display import display, Markdown

langchain.__version__ = '0.0.240'


In [3]:
llm = ChatOpenAI(
    model="gpt-4-0613",
    temperature=0,
)

In [4]:
class OpenAIFunctionTool(Tool):
    openai_function: dict
    def run(self, tool_input, *args, **kwargs):
        return super().run({ "input": tool_input }, *args, **kwargs)

class CustomOpenAIFunctionsAgent(OpenAIFunctionsAgent):
    @property
    def functions(self):
        return [
            t.openai_function if hasattr(t, "openai_function") else dict(format_tool_to_openai_function(t))
            for t in self.tools
        ]

def openapi_spec_to_tools(spec_url: str):

    functions, orig_call_api = openapi_spec_to_openai_fn(OpenAPISpec.from_url(spec_url))

    def create_call_api(name):
        def call_api(*args, **kwargs):
            return orig_call_api(name, *args, **kwargs).text
        return call_api

    return [
        OpenAIFunctionTool(
            name=f_spec["name"],
            func=create_call_api(f_spec["name"]),
            description=f_spec["description"],
            openai_function=f_spec,
        )
        for f_spec in functions
    ]

tools = openapi_spec_to_tools(ELAWS_OPENAPI_SPEC_URI)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [5]:
agent = CustomOpenAIFunctionsAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    system_message=SystemMessage(
        content="You are a helpful and careful AI assistant designed to answer questions about Japanese laws using the given functions. Keep in mind that the exact words and characters matter for laws. Do not make up an answer.",
    )
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
)

In [6]:
query = "デジタル庁設置法（令和三年法律第三十六号）第一条の内容を教えてください。"
output = agent_executor.run(query)



> Entering new AgentExecutor chain...

Invoking: `GetArticles` with `{'params': {'lawNum': '令和三年法律第三十六号', 'article': '第一条'}}`


{"Result":{"Code":"0","Message":""},"ApplData":{"LawId":"","LawNum":"令和三年法律第三十六号","Article":"第一条","Paragraph":"","AppdxTable":"","LawContents":"\n      <Article Num=\"1\">\n        <ArticleCaption>（目的）</ArticleCaption>\n        <ArticleTitle>第一条</ArticleTitle>\n        <Paragraph Num=\"1\">\n          <ParagraphNum/>\n          <ParagraphSentence>\n            <Sentence Num=\"1\" WritingMode=\"vertical\">この法律は、デジタル庁の設置並びに任務及びこれを達成するため必要となる明確な範囲の所掌事務を定めるとともに、その所掌する行政事務を能率的に遂行するため必要な組織に関する事項を定めることを目的とする。</Sentence>\n          </ParagraphSentence>\n        </Paragraph>\n      </Article>\n    "}}デジタル庁設置法（令和三年法律第三十六号）第一条の内容は以下の通りです。

（目的）
第一条
この法律は、デジタル庁の設置並びに任務及びこれを達成するため必要となる明確な範囲の所掌事務を定めるとともに、その所掌する行政事務を能率的に遂行するため必要な組織に関する事項を定めることを目的とする。

> Finished chain.


In [7]:
answer = "この法律は、デジタル庁の設置並びに任務及びこれを達成するため必要となる明確な範囲の所掌事務を定めるとともに、その所掌する行政事務を能率的に遂行するため必要な組織に関する事項を定めることを目的とする。"

display(Markdown("### Query:"))
display(Markdown(f"""\
````
{query}
````
"""))

display(Markdown("### Output:"))
display(Markdown(f"""\
````
{output}
````
"""))

success = answer in output

display(Markdown("### Result:"))

if success:
    display(Markdown("✅ Success: the correct answer included."))
else:
    display(Markdown("❌ Fail: the correct answer not included."))

### Query:

````
デジタル庁設置法（令和三年法律第三十六号）第一条の内容を教えてください。
````


### Output:

````
デジタル庁設置法（令和三年法律第三十六号）第一条の内容は以下の通りです。

（目的）
第一条
この法律は、デジタル庁の設置並びに任務及びこれを達成するため必要となる明確な範囲の所掌事務を定めるとともに、その所掌する行政事務を能率的に遂行するため必要な組織に関する事項を定めることを目的とする。
````


### Result:

✅ Success: the correct answer included.